## Data and Imports

In [1]:
import pickle
import pandas as pd
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn.model_selection import RandomizedSearchCV
import random
from sklearn.metrics import average_precision_score, roc_auc_score

In [2]:
def get_data(dataset,vectorizer):
    '''
    returns feature matrix for specified dataset and vectorizer
    @param dataset: string specifying dataset, "train","dev",etc
    @param vectorizer: string specifying vectorizer "binary","count",etc

    '''
    with open(f'../data/{dataset}_{vectorizer}_subsampled_data.pckl', 'rb') as f:
        return pickle.load(f)


In [3]:
## getting train_y and dev_y
with open('../data/train_labels.pckl', 'rb') as f:
    trainY = pickle.load(f)

with open('../data/dev_labels.pckl', 'rb') as f:
    devY = pickle.load(f)

## Perceptron

In [5]:
from sklearn.linear_model import Perceptron


random.seed(100)


vectorizers = ['count',
               'tfidf',
               #'hashing',
               'binary',
               #'hashing_binary',
               ]

param_grid = {"penalty":["l1","l2","elasticnet",None],"fit_intercept":[True,False],"eta0":sp_rand()}


for vectorizer in vectorizers:
    print("--------------------------")
    print(vectorizer)
    trainX = get_data("train",vectorizer)
    valX = get_data("dev",vectorizer)
    
    clf = Perceptron(early_stopping=True)
    
    rsearch = RandomizedSearchCV(n_jobs=-1, random_state=0, estimator=clf,param_distributions=param_grid)
    
    rsearch.fit(trainX,trainY)
    
    print(rsearch.best_estimator_)
    
    #ytrain_score = rsearch.predict_proba(trainX)
    yval_score = rsearch.decision_function(valX)
    
    
    #print("Train AUC",roc_auc_score(trainY,ytrain_score))
    #print("Train AP",average_precision_score(trainY,ytrain_score))
    print("Val AUC",roc_auc_score(devY,yval_score))
    print("Val AP",average_precision_score(devY,yval_score))
    print("--------------------------")


--------------------------
count
Perceptron(alpha=0.0001, class_weight=None, early_stopping=True,
           eta0=0.9636627605010293, fit_intercept=True, max_iter=1000,
           n_iter_no_change=5, n_jobs=None, penalty='l2', random_state=0,
           shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
           warm_start=False)
Val AUC 0.7060673349928509
Val AP 0.17427758321442435
--------------------------
--------------------------
tfidf
Perceptron(alpha=0.0001, class_weight=None, early_stopping=True,
           eta0=0.4375872112626925, fit_intercept=True, max_iter=1000,
           n_iter_no_change=5, n_jobs=None, penalty='l1', random_state=0,
           shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
           warm_start=False)
Val AUC 0.7394226525165951
Val AP 0.2157023479250944
--------------------------
--------------------------
binary
Perceptron(alpha=0.0001, class_weight=None, early_stopping=True,
           eta0=0.4236547993389047, fit_intercept=Fa

## Linear SVM

In [6]:
from sklearn.svm import LinearSVC
import scipy.stats
param_grid = {"penalty":["l1","l2"],"loss":["squared_hinge"],"C":scipy.stats.reciprocal(a=1e-4,b=1e2)}

for vectorizer in vectorizers:
    print("--------------------------")
    print(vectorizer)
    trainX = get_data("train",vectorizer)
    valX = get_data("dev",vectorizer)
    
    clf = LinearSVC()
    
    rsearch = RandomizedSearchCV(n_jobs=-1, estimator=clf,param_distributions=param_grid)
    
    rsearch.fit(trainX,trainY)
    
    print(rsearch.best_estimator_)
    
    #ytrain_score = rsearch.predict(trainX)
    yval_score = rsearch.decision_function(valX)
    
    
    #print("Train AUC",roc_auc_score(trainY,ytrain_score))
    #print("Train AP",average_precision_score(trainY,ytrain_score))
    print("Val AUC",roc_auc_score(devY,yval_score))
    print("Val AP",average_precision_score(devY,yval_score))
    print("--------------------------")

--------------------------
count
LinearSVC(C=0.00017229584247800946, class_weight=None, dual=True,
          fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
          max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
          tol=0.0001, verbose=0)
Val AUC 0.7507259837160689
Val AP 0.2289730118025493
--------------------------
--------------------------
tfidf


c:\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=0.0484152602408974, class_weight=None, dual=True,
          fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
          max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
          tol=0.0001, verbose=0)
Val AUC 0.7522518462302721
Val AP 0.23281083589105311
--------------------------
--------------------------
binary
LinearSVC(C=0.0004964258537479101, class_weight=None, dual=True,
          fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
          max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
          tol=0.0001, verbose=0)
Val AUC 0.7493476395367485
Val AP 0.23021511582458373
--------------------------


In [9]:
from sklearn.svm import LinearSVC
import scipy.stats
param_grid = {"penalty":["l1","l2"],"loss":["squared_hinge"],"C":scipy.stats.reciprocal(a=1e-4,b=1e2)}

for vectorizer in ['tfidf']:
    print("--------------------------")
    print(vectorizer)
    trainX = get_data("train",vectorizer)
    valX = get_data("dev",vectorizer)
    
    clf = LinearSVC(C=0.18590843630169634, class_weight=None, dual=True,
          fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
          max_iter=5000, multi_class='ovr', penalty='l2', random_state=0,
          tol=0.0001, verbose=0)
    
    #rsearch = RandomizedSearchCV(n_jobs=-1, random_state=0, estimator=clf,param_distributions=param_grid)
    
    clf.fit(trainX,trainY)
    
    #print(rsearch.best_estimator_)
    #print(rsearch.best_estimator_.n_iter_)
    
    #ytrain_score = rsearch.predict(trainX)
    yval_score = clf.decision_function(valX)
    
    
    #print("Train AUC",roc_auc_score(trainY,ytrain_score))
    #print("Train AP",average_precision_score(trainY,ytrain_score))
    print("Val AUC",roc_auc_score(devY,yval_score))
    print("Val AP",average_precision_score(devY,yval_score))
    print("--------------------------")

--------------------------
tfidf


c:\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Val AUC 0.7515879372713236
Val AP 0.23180908887441007
--------------------------


In [10]:
with open('../data/best_svm_tfidf.pckl', 'rb') as f:
    pickle.dump(clf, f)

FileNotFoundError: [Errno 2] No such file or directory: '../data/best_svm_tfidf.pckl'

## Non-Linear SVM

In [ ]:
from sklearn import svm
from sklearn.kernel_approximation import Nystroem
vectorizers = ['count', 'tfidf', 'hashing', 'binary', 'hashing_binary']

for vectorizer in vectorizers:
    for kernel in ["rbf","polynomial"]:
        for n in [1000,5000,10000,25000,45000]:
            print("---------------------------------")
            print(kernel)
            print(vectorizer)
            print(n)
            trainX = get_data("train",vectorizer)
            valX = get_data("dev",vectorizer)
    
            clf = svm.LinearSVC(max_iter=400,tol=1e-2,C=.1)
   
    
            if(kernel=="rbf"):
                feature_map_nystroem = Nystroem(kernel=kernel,n_components=n)
        
            else:
                feature_map_nystroem = Nystroem(kernel=kernel,degree=2.0,n_components=n)
        
            train_transformed = feature_map_nystroem.fit_transform(trainX)
    
            val_transformed = feature_map_nystroem.fit_transform(valX)

    
            clf.fit(train_transformed,trainY)
    
            ytrain_score = clf.predict(train_transformed)
            yval_score = clf.predict(val_transformed)
    
    
            print("Train AUC",roc_auc_score(trainY,ytrain_score))
            print("Train AP",average_precision_score(trainY,ytrain_score))
            print("Val AUC",roc_auc_score(devY,yval_score))
            print("Val AP",average_precision_score(devY,yval_score))
    
            print("-----------------------------------")

    
    


---------------------------------
rbf
count
1000
Train AUC 0.6431891242883149
Train AP 0.5846210445619413
Val AUC 0.48710227133723677
Val AP 0.1000083194710632
-----------------------------------
---------------------------------
rbf
count
5000
Train AUC 0.6435958015415004
Train AP 0.5849188233638523
Val AUC 0.4377501848438885
Val AP 0.093414327577478
-----------------------------------
---------------------------------
rbf
count
10000
